In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
from datacube.storage.storage import write_dataset_to_netcdf
from fc.fractional_cover import fractional_cover
import pandas as pd

dc = datacube.Datacube(app='fc-tester-example')

Populating the interactive namespace from numpy and matplotlib


In [ ]:
scenes = [
    ('ls8','113','082','2013-10-29'),
    ('ls8','111','077','2014-06-28'),
    ('ls8','108','083','2014-09-11'),
    ('ls8','108','079','2014-09-27'),
    ('ls8','105','069','2014-10-08'),
    ('ls8','103','076','2013-06-17'),
    ('ls8','099','079','2015-03-23'),
    ('ls8','098','077','2015-04-01'),
    ('ls8','097','081','2015-09-17'),
    ('ls8','098','071','2015-12-13'),
    ('ls8','095','082','2015-11-06'),
    ('ls8','094','085','2015-11-15'),
    ('ls8','093','086','2015-11-24'),
    ('ls8','096','072','2015-08-25'),
    ('ls8','091','090','2015-10-24'),
    ('ls8','090','086','2015-02-03'),
    ('ls8','090','084','2015-02-19'),
    ('ls8','090','085','2015-07-29'),
    ('ls8','091','079','2015-06-18'),
    ('ls8','089','084','2014-08-20'),
    ('ls8','090','079','2014-08-27'),
    ('ls8','089','079','2014-09-05')]

In [2]:
scenes = [
    ('ls5','089','079','1988-10-15'),
    ('ls5','089','079','1994-09-14'),
    ('ls5','089','079','2004-04-02'),
    ('ls7','089','079','2001-04-18'),
    ('ls5','089','079','1996-06-15'),
    ('ls8','089','079','2014-09-05')]

In [3]:
regression_coefficients = {
    'blue': [0.00041,0.97470],
    'green': [0.00289,0.99779],
    'red': [0.00274,1.00446],
    'nir': [0.00004,0.98906],
    'swir1': [0.00256,0.99467],
    'swir2': [-0.00327,1.02551]
}

In [4]:
measurements = [
    {'name': 'BS',
     'src_var': 'BS',
     'units': "percent",
     'dtype': 'int8',
     'nodata': -1,
     'zlib': True,
     'fletcher32': True,
     'attrs':{
         'long_name': "Bare ground fraction percentage",
         'comment': "Fraction of bare ground including rock, bare and disturbed soil, within the Landsat pixel as percentages",
         'alias': "bare",
         'coverage_content_type': 'modelResult'
     },
      'aliases':["bare"]},
    {'name': 'PV',
      'src_var': 'PV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Green cover fraction percentage",
          'comment': "Fraction of green cover including green groundcover and green leaf material over all strata, within the Landsat pixel, as percentages",
          'alias': "green",
          'coverage_content_type': 'modelResult'
          },
      'aliases': ["green_veg"]},
    {'name': 'NPV',
      'src_var': 'NPV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Non-green cover fraction percentage",
          'comment': "Fraction of non green cover including litter, dead leaf and branches over all strata, within the Landsat pixel, as percentages",
          'alias': "dead",
          'coverage_content_type': 'modelResult'
         },
      'aliases': ["dead_veg"]},
    {'name': 'UE',
      'src_var': 'UE',
      'units': "1",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Unmixing error",
          'comment': "Eclidean Norm of the Residual Vector",
          'alias': "err",
          'coverage_content_type': 'qualityInformation'
          },
      'aliases': ["err"]}
]

In [7]:
def write_old_and_new_fc(sensor, path, row, date):
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')    
    target_dir = '/g/data1/r78/dg6911/jrsrp-fc/'+path+'_'+row+'/'
    print('path: ' + path + ' row:' + row + ' date:' + date)    
    print('target dir: '+target_dir)
    date_nodash = date.replace('-','')
    target_file = '_'+sensor+'_'+path+'_'+row+'_'+date_nodash+'.nc'
    #old_target = target_dir + 'old_fc' + target_file
    #nbart_target = target_dir + 'nbart' + target_file
    new_target = target_dir + 'new_end_fc' + target_file
    
    source_filter = dict(product=sensor+'_nbart_scene', sat_path=path, sat_row=row)
#     try:
#         print('reading old FC')
#         data = dc.load(product=sensor+'_fc_albers', time=(day_before, day_after), source_filter=source_filter)
    
#         print('writing old FC')
#         write_dataset_to_netcdf(data, old_target)
#     except:
#         print('could not find: '+sensor+'_p'+path+'_r'+row+'_'+date)
#         print('error:', sys.exc_info()[0])
    
    print('reading NBART')
    data = dc.load(product=sensor+'_nbart_albers', time=(day_before, day_after), source_filter=source_filter)
    #write_dataset_to_netcdf(data, nbart_target)
    if(sensor == 'ls8'):
         use_coefficients=regression_coefficients
    else:
         use_coefficients=None
    print('calculating new FC')
    data = data.squeeze()
    new_fc_data = fractional_cover(nbar_tile=data, measurements=measurements, regression_coefficients=use_coefficients)
    
    print('writing new FC')
    write_dataset_to_netcdf(new_fc_data, new_target)
    

In [8]:
for scene in scenes:
    write_old_and_new_fc(*scene)

path: 089 row:079 date:1988-10-15
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
path: 089 row:079 date:1994-09-14
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
path: 089 row:079 date:2004-04-02
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
path: 089 row:079 date:2001-04-18
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
path: 089 row:079 date:1996-06-15
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
path: 089 row:079 date:2014-09-05
target dir: /g/data1/r78/dg6911/jrsrp-fc/089_079/
reading NBART
calculating new FC
writing new FC
